In [1]:


import sparknlp
 
spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



23/01/20 09:22:13 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
23/01/20 09:22:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-gpu_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bbcccd70-fab0-49cd-a478-59b982a4bffd;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.project

23/01/20 09:22:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/20 09:22:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/20 09:22:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:
import pandas as pd
import openpyxl
import numpy as np
def sentancebreak(breaking, sh):
    check = True
    broken = breaking.split(' ')
    store = []
    for l in broken:
        check = True
        for i in range(1,sh.max_row+1):
            keyword = sh.cell(row=i,column=1).value
            if(l == keyword):
                print("keyword check")
                for j in range(1,sh.max_column+1):
                   
                    print("one check")
                    one = sh.cell(row=i,column=j).value
                    print(one)
                    if(one == 1):
                        check = False
                        print("store check")
                        store.append(keyword+" 0 0 "+sh.cell(row=1,column=j).value)
                        break
                break
                        

        if(check==True):
            store.append(l+" 0 0 0")

    store.append(". O\n")

    return store


def adding(base, add):
    for k in add:
        base.append(k)
    return base
        
df = pd.read_excel("./Keyword Map NLP.xlsx")
Loc = df.loc[df['Loc']==1]
#print(Loc)
workbook = openpyxl.load_workbook('./Keyword Map NLP.xlsx')
sh = workbook.active
get= []
print("start")
for i in range(1,sh.max_column+1):
    #print(i)
    search = sh.cell(row=1,column=i).value
    #print(search)
    if(search == 'Loc'):
        for j in range(2,sh.max_row+1):
            one = sh.cell(row=j,column=i).value
            if(one == 1):
                #print(sh.cell(row=j,column=1).value)
                get.append(sh.cell(row=j,column=1).value)
        break

sentances = ["-DOCSTART- -X- -X- O\n"]
#file = open("read.txt",'w')

for i in range(1,sh.max_row+1):
    keyword = sh.cell(row=i,column=1).value
    for j in range(1,sh.max_column+1):
        one = sh.cell(row=i,column=j).value
        if(one == 1):
            sentances.append(keyword+" 0 0 "+sh.cell(row=1,column=j).value+"\n")
            break

for w in get:
    filer = (f"i want to go to the {w}")
    temp = sentancebreak(filer,sh)
    adding(sentances,temp)
for w in get:
    filer = (f"navigate to {w}")
    temp = sentancebreak(filer,sh)
    sentances=adding(sentances,temp)

filer = (f"i want to make coffee")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"where am i")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"what can i do here")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)


sentances=adding(sentances,temp)
filer = (f"prepare meal")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"use refrigerator")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"use fan")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"use oven")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"use stove")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"i would like to wash sheets")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"i would like to watch television")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"wash sheets")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"wash clothes")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"cook dinner")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"i would like to cook dinner")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"make food")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"i want to read a book")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"open camera")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"open cabinet")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"start camera")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"what can i do here")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"where is the coffee")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"rewind video")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"pause video")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"skip forward")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"negative")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"positive")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"yes")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"no")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello yes")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello no")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"shutdown app")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"shut")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"play video")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello shut down app")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello shut")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello play video")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello negative")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello positive")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"correct")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"incorrect")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello correct")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello incorrect")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)


filer = (f"i want to make coffee")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"where am i")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"what can i do here")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)


for w in get:
    filer = (f"hello i want to go to the {w}")
    temp = sentancebreak(filer,sh)
    adding(sentances,temp)
for w in get:
    filer = (f"hello navigate to {w}")
    temp = sentancebreak(filer,sh)
    sentances=adding(sentances,temp)
filer = (f"hello prepare meal")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello use refrigerator")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello use fan")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello use oven")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello use stove")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello i would like to wash sheets")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello i would like to watch television")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello wash sheets")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"hello wash clothes")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"hello cook dinner")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello i would like to cook dinner")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello make food")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello i want to read a book")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"hello open camera")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello start camera")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"hello what can i do here")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello where is the coffee")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello location")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"location")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello activity")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"activity")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

with open('eng_custom.train', 'w') as file:
      

    file.writelines("% s\n" % data for data in sentances)



sentances = ["-DOCSTART- -X- -X- O\n"]
#file = open("read.txt",'w')



for w in get:
    filer = (f"hello i want to go to the {w}")
    temp = sentancebreak(filer,sh)
    adding(sentances,temp)
for w in get:
    filer = (f"hello navigate to {w}")
    temp = sentancebreak(filer,sh)
    sentances=adding(sentances,temp)

filer = (f"hello i want to make coffee")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello where am i")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello what can i do here")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello prepare meal")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello use refrigerator")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello start camera")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)

filer = (f"hello correct")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello incorrect")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)


filer = (f"hello play video")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello negative")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello positive")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)


filer = (f"hello yes")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello no")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello location")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello location.")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
filer = (f"hello activity")
temp = sentancebreak(filer,sh)
sentances=adding(sentances,temp)
with open('eng_custom.testa', 'w') as file:
      

    file.writelines("% s\n" % data for data in sentances)





start
keyword check
one check
want
one check
0
one check
0
one check
0
one check
0
one check
0
one check
0
one check
1
store check
keyword check
one check
go
one check
0
one check
1
store check
keyword check
one check
location
one check
1
store check
keyword check
one check
want
one check
0
one check
0
one check
0
one check
0
one check
0
one check
0
one check
1
store check
keyword check
one check
go
one check
0
one check
1
store check
keyword check
one check
kitchen
one check
1
store check
keyword check
one check
want
one check
0
one check
0
one check
0
one check
0
one check
0
one check
0
one check
1
store check
keyword check
one check
go
one check
0
one check
1
store check
keyword check
one check
livingroom
one check
1
store check
keyword check
one check
want
one check
0
one check
0
one check
0
one check
0
one check
0
one check
0
one check
1
store check
keyword check
one check
go
one check
0
one check
1
store check
keyword check
one check
bedroom
one check
1
store check
keyword check


In [3]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [4]:
import pyspark.sql.functions as F

#training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
#        F.expr("cols['1']").alias("pos"),
#        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


In [5]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [6]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(500)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    


ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]


In [7]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

2023-01-20 09:22:25.973814: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-20 09:22:25.973832: I external/org_tensorflow/tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-20 09:22:26.490042: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 09:22:26.490640: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open 

Training started - total epochs: 500 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 24 - training examples: 265
Epoch 1/500 started, lr: 0.003, dataset size: 265
Epoch 1/500 - 0.86s - loss: 44.018494 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.19s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 0	 14	 0.0	 0.0	 0.0
Loc	 14	 0	 11	 1.0	 0.56	 0.71794873
Act	 19	 3	 17	 0.8636364	 0.5277778	 0.6551724
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 54	 32	 0	 0.627907	 1.0	 0.7714285
Obj	 20	 9	 0	 0.6896552	 1.0	 0.81632656
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 12	 0	 0	 1.0	 1.0	 1.0
tp: 119 fp: 44 fn: 44 labels: 8
Macro-average	 prec: 0.5226498, rec: 0.51097226, f1: 0.5167451
Micro-average	 prec: 0.73006135, rec: 0.73006135, f1: 0.73006135
Epoch 2/500 started, lr: 0.0029850747, dataset size: 265
Epoch 2/500 - 0.18s - loss: 18.838629 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
labe

Epoch 14/500 - 0.17s - loss: 0.44127014 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 2	 1.0	 0.92	 0.9583333
Act	 34	 2	 2	 0.9444444	 0.9444444	 0.9444444
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0	 1.0	 1.0
Obj	 18	 4	 2	 0.8181818	 0.9	 0.8571428
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 12	 0	 0	 1.0	 1.0	 1.0
tp: 157 fp: 6 fn: 6 labels: 8
Macro-average	 prec: 0.97032833, rec: 0.9705556, f1: 0.970442
Micro-average	 prec: 0.9631902, rec: 0.9631902, f1: 0.96319014
Epoch 15/500 started, lr: 0.0028037382, dataset size: 265
Epoch 15/500 - 0.17s - loss: 0.40059054 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 2	 1.0	 0.92	 0.9583333
Act	 35	 3	 1	 0.92105263	 0.9722222	 0.945946
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	

Epoch 28/500 - 0.17s - loss: 0.117879905 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 24	 0	 1	 1.0	 0.96	 0.9795918
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0	 1.0	 1.0
Obj	 19	 3	 1	 0.8636364	 0.95	 0.9047619
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 12	 0	 0	 1.0	 1.0	 1.0
tp: 159 fp: 4 fn: 4 labels: 8
Macro-average	 prec: 0.9793832, rec: 0.98180556, f1: 0.98059285
Micro-average	 prec: 0.9754601, rec: 0.9754601, f1: 0.9754601
Epoch 29/500 started, lr: 0.002631579, dataset size: 265
Epoch 29/500 - 0.17s - loss: 0.080858275 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 2	 1.0	 0.92	 0.9583333
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 

Epoch 41/500 - 0.17s - loss: 0.122336216 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 25	 0	 0	 1.0	 1.0	 1.0
Act	 35	 1	 1	 0.9722222	 0.9722222	 0.9722222
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0	 1.0	 1.0
Obj	 19	 1	 1	 0.95	 0.95	 0.95
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 12	 0	 0	 1.0	 1.0	 1.0
tp: 161 fp: 2 fn: 2 labels: 8
Macro-average	 prec: 0.99027777, rec: 0.99027777, f1: 0.99027777
Micro-average	 prec: 0.9877301, rec: 0.9877301, f1: 0.9877301
Epoch 42/500 started, lr: 0.0024896264, dataset size: 265
Epoch 42/500 - 0.16s - loss: 0.11826258 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 25	 0	 0	 1.0	 1.0	 1.0
Act	 35	 1	 1	 0.9722222	 0.9722222	 0.9722222
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0	 1.0	 1.0
Obj	 

...

Epoch 469/500 - 0.16s - loss: 0.04375054 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 25	 0	 0	 1.0	 1.0	 1.0
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0	 1.0	 1.0
Obj	 19	 2	 1	 0.9047619	 0.95	 0.92682934
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 12	 0	 0	 1.0	 1.0	 1.0
tp: 160 fp: 3 fn: 3 labels: 8
Macro-average	 prec: 0.98452383, rec: 0.98680556, f1: 0.98566335
Micro-average	 prec: 0.9815951, rec: 0.9815951, f1: 0.9815951
Epoch 470/500 started, lr: 8.9686096E-4, dataset size: 265
Epoch 470/500 - 0.17s - loss: 0.047500785 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 25	 0	 0	 1.0	 1.0	 1.0
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0

Epoch 483/500 - 0.17s - loss: 0.03707613 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 24	 0	 1	 1.0	 0.96	 0.9795918
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0	 1.0	 1.0
Obj	 19	 3	 1	 0.8636364	 0.95	 0.9047619
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 12	 0	 0	 1.0	 1.0	 1.0
tp: 159 fp: 4 fn: 4 labels: 8
Macro-average	 prec: 0.9793832, rec: 0.98180556, f1: 0.98059285
Micro-average	 prec: 0.9754601, rec: 0.9754601, f1: 0.9754601
Epoch 484/500 started, lr: 8.784773E-4, dataset size: 265
Epoch 484/500 - 0.17s - loss: 0.067130454 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 24	 0	 1	 1.0	 0.96	 0.9795918
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54

Epoch 497/500 - 0.17s - loss: 0.03685187 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 25	 0	 0	 1.0	 1.0	 1.0
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0	 1.0	 1.0
Obj	 19	 2	 1	 0.9047619	 0.95	 0.92682934
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 12	 0	 0	 1.0	 1.0	 1.0
tp: 160 fp: 3 fn: 3 labels: 8
Macro-average	 prec: 0.98452383, rec: 0.98680556, f1: 0.98566335
Micro-average	 prec: 0.9815951, rec: 0.9815951, f1: 0.9815951
Epoch 498/500 started, lr: 8.6083217E-4, dataset size: 265
Epoch 498/500 - 0.16s - loss: 0.051413838 - batches: 9
Quality on validation dataset (20.0%), validation examples = 53
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 14	 0	 0	 1.0	 1.0	 1.0
Loc	 25	 0	 0	 1.0	 1.0	 1.0
Act	 34	 1	 2	 0.9714286	 0.9444444	 0.95774645
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 54	 0	 0	 1.0

CPU times: user 257 ms, sys: 115 ms, total: 372 ms
Wall time: 1min 35s


2023-01-20 09:23:59.280778: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [8]:

#predictions = ner_model.transform(testing_data) 
#predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#           .select(F.expr("cols['0']").alias("token"),
#                   F.expr("cols['1']").alias("ground_truth"),
#                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [9]:
from sklearn.metrics import classification_report
 
#preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#                      .select(F.expr("cols['0']").alias("token"),
#                              F.expr("cols['1']").alias("ground_truth"),
#                              F.expr("cols['2']").alias("prediction")).toPandas()
 
#print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

In [10]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [11]:
text_list = [
    """Navagate to kitchen"""
]


In [12]:
loaded_ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


2023-01-20 09:23:59.972882: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [13]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_chunk': ['navigate to kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['Act', '0', 'Loc'], 'embeddings': ['navigate', 'to', 'kitchen']}
['Act', '0', 'Loc']


In [14]:
light.annotate("Navigate to kitchen")


{'document': ['Navigate to kitchen'],
 'ner_chunk': ['Navigate to kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'embeddings': ['Navigate', 'to', 'kitchen']}

In [18]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

Enter Testing Text
i want to go to the living room
['0', 'Rqt', '0', 'Act', '0', '0', 'Loc', 'Loc']
Enter New Text
 put coffe on shelf
['Act', 'Obj', 'Act', 'Obj']
Enter New Text
shelf
['Obj']
Enter New Text
shelve
['Act']
Enter New Text
exit


In [16]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

In [17]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




